In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install import-ipynb

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
import pickle
import time
import sys

import import_ipynb
import AttackDetectionModel

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
importing Jupyter notebook from AttackDetectionModel.ipynb
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-10-03 21:33:45.864151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 21:33:45.981833: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 21:33:45.981849: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-03 21:33:46.002089: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 21:33:46.475024: W tensorflow/stream_executor/pla

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-10-03 21:33:46.950922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 21:33:46.987366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 21:33:46.987511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-03 21:33:46.987631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared obj

# Data Pre-Processing

In [3]:
# Get benign traffic 

df_dd_benign = AttackDetectionModel.get_files('../../nbaiot/Danmini_Doorbell')
#print(df_dd_benign.shape)

# Get Bashlite traffic

df_dd_bashlite = AttackDetectionModel.get_files('../../nbaiot/Danmini_Doorbell/gafgyt')
#print(df_dd_bashlite.shape)

# Get Mirai traffic

df_dd_mirai = AttackDetectionModel.get_files('../../nbaiot/Danmini_Doorbell/mirai')
#print(df_dd_mirai.shape)

In [4]:
# Normalize with the min-max scaler

scaler = MinMaxScaler()
df_dd_benign_norm = scaler.fit_transform(df_dd_benign)
df_dd_bashlite_norm = scaler.fit_transform(df_dd_bashlite)
df_dd_mirai_norm = scaler.fit_transform(df_dd_mirai)

# Sets

In [5]:
# Train set

len_benign_train = int(0.7 * len(df_dd_benign_norm))
X_train_VAE = df_dd_benign_norm[:len_benign_train]
#print(X_train_VAE.shape)

In [6]:
# Test set

X_test_benign = df_dd_benign_norm[len_benign_train:]
#print(X_test_benign.shape)

X_test_VAE = np.concatenate([X_test_benign, df_dd_bashlite_norm, df_dd_mirai_norm])
#print(X_test_VAE.shape)

Y_test_VAE = np.ones(len(X_test_VAE))
Y_test_VAE[:len(X_test_benign)] = 0

# Model

In [7]:
# Parameters

original_dim = X_train_VAE.shape[1]
input_shape = (original_dim,)
intermediate_dim = int(original_dim / 2)
latent_dim = int(original_dim / 3)

# Baseline values
epochs = 50
learning_rate = 0.0001
batch_size = 50
anomaly_threshold = 0.042
number_features = 115

# Dictionary
dict_params = { 'learning_rate': learning_rate, 'batch_size': int(32 * batch_size), 'epochs': int(epochs)}
pbounds = { 'learning_rate': (0.000001, 0.1), 'batch_size': (1, 4.001), 'epochs': (100, 1000)}

In [8]:
# KL Loss function

def vae_loss(x, x_decoded_mean):
    # Compute the average MSE error, then scale it up (sum on all axes)
    
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean))
    
    # Compute the KL loss
    
    kl_loss = - 0.5 * K.sum(1 + z_var - K.square(z_mean) - K.square(K.exp(z_var)), axis=-1)
    
    # Return the average loss over all 
    
    total_loss = K.mean(reconstruction_loss + kl_loss) # Total_loss = reconstruction_loss + kl_loss 
    return total_loss

# (1) Reconstruction Loss - Forces the encoder to generate latent features that minimize the reconstruction error, or else is
# penalized
# (2) KL Loss - Forces the distribution generated by the encoder to be similar to the prior probability of the input vector, 
# pushing latent feature space to normality

In [9]:
# Encoder

inputs, encoder, z_var, z_mean = AttackDetectionModel.vae_encoder(
    input_shape, 
    intermediate_dim, 
    latent_dim, 
    AttackDetectionModel.reparametrization)

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 115)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 57)           6612        ['encoder_input[0][0]']          
                                                                                                  
 z_mean (Dense)                 (None, 38)           2204        ['dense[0][0]']                  
                                                                                                  
 z_var (Dense)                  (None, 38)           2204        ['dense[0][0]']                  
                                                                                            

In [10]:
# Decoder

decoder = AttackDetectionModel.vae_decoder(
    intermediate_dim,
    latent_dim,
    original_dim)

outputs = decoder(encoder(inputs))

decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 38)]              0         
                                                                 
 dense_1 (Dense)             (None, 57)                2223      
                                                                 
 dense_2 (Dense)             (None, 115)               6670      
                                                                 
Total params: 8,893
Trainable params: 8,893
Non-trainable params: 0
_________________________________________________________________


# Train

In [11]:
# Apply Bayesian optimization to choose the best hyperparameters

optimizer = BayesianOptimization(
    f = AttackDetectionModel.maximize_training(X_train_VAE, 
                                               X_test_VAE, 
                                               Y_test_VAE, 
                                               inputs, 
                                               outputs, 
                                               vae_loss, 
                                               dict_params, 
                                               anomaly_threshold),
    pbounds = pbounds,
    verbose = 1, 
    random_state = 1,
)

opt_start = time.time()

optimizer.maximize(init_points = 5, n_iter = 5)

opt_end = time.time()
opt_time = opt_end - opt_start
print("Optimization time:", train_time)

print(optimizer.max)

Train on 34683 samples
Epoch 1/50


2022-10-03 21:34:04.368348: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-03 21:34:04.382549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


34683/34683 [==============================] - 0s 14us/sample - loss: 33118.9621
Epoch 2/50
34683/34683 [==============================] - 0s 6us/sample - loss: 31416.3163
Epoch 3/50
34683/34683 [==============================] - 0s 6us/sample - loss: 29589.9576
Epoch 4/50
34683/34683 [==============================] - 0s 6us/sample - loss: 27678.9868
Epoch 5/50
34683/34683 [==============================] - 0s 7us/sample - loss: 25496.8164
Epoch 6/50
34683/34683 [==============================] - 0s 6us/sample - loss: 22793.8068
Epoch 7/50
34683/34683 [==============================] - 0s 6us/sample - loss: 19394.6848
Epoch 8/50
34683/34683 [==============================] - 0s 6us/sample - loss: 15429.2784
Epoch 9/50
34683/34683 [==============================] - 0s 6us/sample - loss: 11350.7383
Epoch 10/50
34683/34683 [==============================] - 0s 6us/sample - loss: 7860.5745
Epoch 11/50
34683/34683 [==============================] - 0s 6us/sample - loss: 5321.8914
Epoch 12/

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


|   iter    |  target   | batch_... |  epochs   | learni... |
-------------------------------------------------------------


TypeError: 'numpy.float64' object is not callable

In [ ]:
# Training step with the best hyperparameters

opt = optimizers.Adam(learning_rate = optimizer.max['params']['learning_rate'], clipvalue = 0.5)

vae_model = Model(inputs, outputs, name = 'vae_mlp')
vae_model.compile(optimizer = opt, loss = vae_loss)
vae_model.summary()

train_start = time.time()

history = vae_model.fit(X_train_VAE, X_train_VAE,
                            shuffle = True,
                            epochs = int(optimizer.max['params']['epochs']),
                            batch_size = int(32 * optimizer.max['params']['batch_size']))

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)